In [1]:
import pandas as pd
import numpy as np

# 1. Read your CSV
df = pd.read_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_two_column_with_rank_11_withoutDT.csv")

# 2. Compute the constant denominator: 11 * 0.95 * √3
den = 11 * 0.95 * np.sqrt(3)

# 3. Create the new column
df["DT_LOAD_CAL"] = df["DT_LOAD"] / den

# 4. (Optional) inspect first few rows
print(df[["DT_LOAD", "DT_LOAD_CAL"]].head())

# 5. Save to a new CSV (or overwrite)
df.to_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv", index=False)


      DT_LOAD  DT_LOAD_CAL
0  127.425882     7.040131
1  233.628927    12.907725
2  134.062123     7.406775
3  245.587090    13.568399
4  364.485990    20.137424


MAP THE LOAD OF MEANHLY AVERGAE WITH THE FEEDER

In [ ]:
import pandas as pd

# 1) Load your files
sw = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv")       # contains SWNO, Month_01…Month_12
feeder = pd.read_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv")      # contains feeder_id, DT_LOAD_CAL

# 2) Build a lookup from feeder_id → DT_LOAD_CAL
dt_map = feeder.set_index("FEEDER_ID")["DT_LOAD_CAL"].to_dict()

# 3) Prepare for the walk:
#    We'll keep track of the "current" DT_LOAD_CAL and the last calculated values for each month.
current_dt = None
prev_vals = {f"Month_{i:02d}": None for i in range(1,13)}

# 4) Iterate rows in order, computing chain‐subtract columns:
out_rows = []
for _, row in sw.iterrows():
    swno = row["SWNO"]
    # If this SWNO is a feeder anchor, reset current_dt and prev_vals
    if swno in dt_map:
        current_dt = dt_map[swno]
        # initialize prev_vals for this anchor: first subtraction
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = row[m] - current_dt
    else:
        # continue chain: subtract the same current_dt from last prev_vals
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = prev_vals[m] - current_dt

    # build output row
    out = row.to_dict()
    # add the new _CAL columns
    for i in range(1,13):
        m = f"Month_{i:02d}"
        out[f"{m}_CAL"] = prev_vals[m]
    out_rows.append(out)

# 5) Assemble and write out
out_df = pd.DataFrame(out_rows)
out_df.to_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/feeder_11Kv_sw_with_chain_calculations.csv", index=False)


COMMULATIVE DIFFERENCE FOR MONTHLY AVERAGE LOAD OF SCADA


In [3]:
import pandas as pd

# File paths
feeder_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv"
monthly_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv"
output_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_feeder_11kv_monthly_cumulative_diff.csv"


# Load data
feeder_df = pd.read_csv(feeder_path)
monthly_df = pd.read_csv(monthly_path)

# Merge on FEEDER_ID <-> SWNO
merged = feeder_df.merge(monthly_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

month_cols = [f'Month_{i:02d}' for i in range(1, 13)]

result_rows = []

for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
    group = group.copy()
    group['__orig_idx'] = group.index
    ranks_in_order = group['RANK'].drop_duplicates().tolist()

    dt_load_sums = {
        rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
        for rank in ranks_in_order
    }

    prev_cum = {m: None for m in month_cols}

    for rank in ranks_in_order:
        this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
        first = True
        for idx, row in this_rank_rows.iterrows():
            base = {col: row[col] for col in feeder_df.columns}
            for m in month_cols:
                month_val = row.get(m, None)
                if first:
                    # Only for the first row of this rank
                    if rank == ranks_in_order[0]:
                        cum_val = month_val - dt_load_sums[rank] if pd.notna(month_val) else None
                    else:
                        prev = prev_cum[m]
                        cum_val = prev - dt_load_sums[rank] if prev is not None else None
                    prev_cum[m] = cum_val
                else:
                    cum_val = None  # For subsequent rows at same rank, leave blank
                base[f"{m}_cumulative_diff"] = cum_val
            result_rows.append(base)
            first = False

result_df = pd.DataFrame(result_rows)
output_cols = list(feeder_df.columns) + [f"{m}_cumulative_diff" for m in month_cols]
result_df.to_csv(output_path, index=False, columns=output_cols)
print(f"Saved to {output_path}")


Saved to /media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_feeder_11kv_monthly_cumulative_diff.csv


AFTER THE RANKING AND USE ONLY MAIN CHAIN DONT USING SUBCHAIN

In [1]:
import pandas as pd
import numpy as np

# 1. Read your CSV
df = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/11_WITHOUTDT_CONNECTED_RANKED.csv")

# 2. Compute the constant denominator: 11 * 0.95 * √3
den = 11 * 0.95 * np.sqrt(3)

# 3. Create the new column
df["DT_LOAD_CAL"] = df["DT_LOAD"] / den

# 4. (Optional) inspect first few rows
print(df[["DT_LOAD", "DT_LOAD_CAL"]].head())

# 5. Save to a new CSV (or overwrite)
df.to_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv", index=False)


      DT_LOAD  DT_LOAD_CAL
0  616.320000    34.050959
1  674.080488    37.242158
2  473.442574    26.157148
3  148.855367     8.224085
4  400.698507    22.138124


In [3]:
import pandas as pd

# 1) Load your files
sw = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv")       # contains SWNO, Month_01…Month_12
feeder = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv")      # contains feeder_id, DT_LOAD_CAL

# 2) Build a lookup from feeder_id → DT_LOAD_CAL
dt_map = feeder.set_index("FEEDER_ID")["DT_LOAD_CAL"].to_dict()

# 3) Prepare for the walk:
#    We'll keep track of the "current" DT_LOAD_CAL and the last calculated values for each month.
current_dt = None
prev_vals = {f"Month_{i:02d}": None for i in range(1,13)}

# 4) Iterate rows in order, computing chain‐subtract columns:
out_rows = []
for _, row in sw.iterrows():
    swno = row["SWNO"]
    # If this SWNO is a feeder anchor, reset current_dt and prev_vals
    if swno in dt_map:
        current_dt = dt_map[swno]
        # initialize prev_vals for this anchor: first subtraction
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = row[m] - current_dt
    else:
        # continue chain: subtract the same current_dt from last prev_vals
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = prev_vals[m] - current_dt

    # build output row
    out = row.to_dict()
    # add the new _CAL columns
    for i in range(1,13):
        m = f"Month_{i:02d}"
        out[f"{m}_CAL"] = prev_vals[m]
    out_rows.append(out)

# 5) Assemble and write out
out_df = pd.DataFrame(out_rows)
out_df.to_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/feeder_11Kv_sw_with_chain_calculations.csv", index=False)


In [1]:
import pandas as pd

# File paths (change as needed)
feeder_path = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv"
daily_path_1 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/daily_SWNO_matrix_11KV_YEAR2025.csv"
daily_path_2 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/daily_SWNO_matrix_11KV_YEAR2024.csv"
output_path_1 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv"
output_path_2 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv"


In [2]:
feeder_df = pd.read_csv(feeder_path)

def is_main_chain_only(ranks):
    return all('.' not in str(r) for r in ranks)

main_chain_flags = (
    feeder_df.groupby('FEEDER_ID')['RANK']
    .apply(is_main_chain_only)
    .rename('IS_MAIN_CHAIN_ONLY')
    .reset_index()
)
main_chain_ids = main_chain_flags[main_chain_flags['IS_MAIN_CHAIN_ONLY']]['FEEDER_ID'].tolist()
main_chain_feeder_df = feeder_df[feeder_df['FEEDER_ID'].isin(main_chain_ids)].copy()


In [3]:
def process_year(main_chain_feeder_df, daily_path, output_path):
    daily_df = pd.read_csv(daily_path)
    day_cols = [col for col in daily_df.columns if col.startswith("Day_")]

    merged = main_chain_feeder_df.merge(daily_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

    result_rows = []

    for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
        group = group.copy()
        group['__orig_idx'] = group.index
        ranks_in_order = group['RANK'].drop_duplicates().tolist()

        dt_load_sums = {
            rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
            for rank in ranks_in_order
        }

        prev_cum = {d: None for d in day_cols}

        for rank in ranks_in_order:
            this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
            first = True
            for idx, row in this_rank_rows.iterrows():
                base = {col: row[col] for col in feeder_df.columns}
                for d in day_cols:
                    day_val = row.get(d, None)
                    if first:
                        if rank == ranks_in_order[0]:
                            cum_val = day_val - dt_load_sums[rank] if pd.notna(day_val) else None
                        else:
                            prev = prev_cum[d]
                            cum_val = prev - dt_load_sums[rank] if prev is not None else None
                        prev_cum[d] = cum_val
                    else:
                        cum_val = None
                    base[f"{d}_cumulative_diff"] = cum_val
                result_rows.append(base)
                first = False

    result_df = pd.DataFrame(result_rows)
    output_cols = list(feeder_df.columns) + [f"{d}_cumulative_diff" for d in day_cols]
    result_df.to_csv(output_path, index=False, columns=output_cols)
    print(f"Saved to {output_path}")

# Process each year
process_year(main_chain_feeder_df, daily_path_1, output_path_1)
process_year(main_chain_feeder_df, daily_path_2, output_path_2)


Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv
Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv


In [4]:
import pandas as pd

# --- Identify only main-chain feeders (no sub-branches) ---
feeder_df = pd.read_csv(feeder_path)

def is_main_chain_only(ranks):
    return all('.' not in str(r) for r in ranks)

main_chain_flags = (
    feeder_df.groupby('FEEDER_ID')['RANK']
    .apply(is_main_chain_only)
    .rename('IS_MAIN_CHAIN_ONLY')
    .reset_index()
)
main_chain_ids = main_chain_flags[main_chain_flags['IS_MAIN_CHAIN_ONLY']]['FEEDER_ID'].tolist()
main_chain_feeder_df = feeder_df[feeder_df['FEEDER_ID'].isin(main_chain_ids)].copy()

# --- Processing function ---
def process_year(main_chain_feeder_df, daily_path, output_path):
    daily_df = pd.read_csv(daily_path)
    day_cols = [col for col in daily_df.columns if col.startswith("Day_")]

    merged = main_chain_feeder_df.merge(daily_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

    result_rows = []

    for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
        group = group.copy()
        group['__orig_idx'] = group.index
        ranks_in_order = group['RANK'].drop_duplicates().tolist()

        dt_load_sums = {
            rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
            for rank in ranks_in_order
        }

        prev_cum = {d: None for d in day_cols}

        for rank in ranks_in_order:
            this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
            first = True
            for idx, row in this_rank_rows.iterrows():
                base = {col: row[col] for col in main_chain_feeder_df.columns}
                for d in day_cols:
                    day_val = row.get(d, None)
                    if first:
                        if rank == ranks_in_order[0]:
                            cum_val = day_val - dt_load_sums[rank] if pd.notna(day_val) else None
                        else:
                            prev = prev_cum[d]
                            cum_val = prev - dt_load_sums[rank] if prev is not None else None
                        prev_cum[d] = cum_val
                    else:
                        cum_val = None
                    base[f"{d}_cumulative_diff"] = cum_val
                result_rows.append(base)
                first = False

    result_df = pd.DataFrame(result_rows)
    # --- Add FROM_SWITCH and TO_SWITCH columns ---
    if "FROM_TO" in result_df.columns:
        from_to_split = result_df["FROM_TO"].str.split("-", n=1, expand=True)
        result_df["FROM_SWITCH"] = from_to_split[0]
        result_df["TO_SWITCH"] = from_to_split[1]
        cols = list(result_df.columns)
        i = cols.index("FROM_TO") + 1
        for col in ["FROM_SWITCH", "TO_SWITCH"]:
            cols.insert(i, cols.pop(cols.index(col)))
            i += 1
        result_df = result_df[cols]
    # --- Save ---
    output_cols = list(result_df.columns)  # Save all columns, now includes FROM_SWITCH and TO_SWITCH
    result_df.to_csv(output_path, index=False, columns=output_cols)
    print(f"Saved to {output_path}")

# --- Process each year ---
process_year(main_chain_feeder_df, daily_path_1, output_path_1)
process_year(main_chain_feeder_df, daily_path_2, output_path_2)


Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv
Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv


NEW LOGIC TO CALCULATE THE LOAD

In [28]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Per-node monthly loads (integer) + cumulative section loads (integer, sec_cum_* only)
using monthly_SWNO_matrix_11KV.csv (SWNO == FEEDER_ID).

Output columns added:
  - SUM_DT_LOAD_CAL, FRACTION
  - feeder_2024_01.._12      (float)
  - node_load_2024_01.._12   (Int64, rounded BEFORE cumulative)
  - sec_cum_2024_01.._12     (Int64)

Writes:
  /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv
"""

from __future__ import annotations
from pathlib import Path
import numpy as np
import pandas as pd

# ── Paths ───────────────────────────────────────────────────────────────────
BASE      = "/media/sagark24/New Volume/MERGE CDIS"
IN_FEDDER = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/11_WITHOUTDT_CONNECTED_RANKED.csv")
IN_MATRIX = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/monthly_SWNO_matrix_11KV.csv")
OUT_FILE  = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv")

# ── Months & columns ────────────────────────────────────────────────────────
YEAR = 2024
MONTH_KEYS = [f"Month_{i:02d}" for i in range(1, 13)]          # in matrix
MONTH_TAGS = [f"{YEAR}-{i:02d}" for i in range(1, 13)]         # long tags
NODE_COLS  = [f"node_load_{YEAR}_{i:02d}" for i in range(1, 13)]
CUM_COLS   = [f"sec_cum_{YEAR}_{i:02d}"   for i in range(1, 13)]
FEED_COLS  = [f"feeder_{YEAR}_{i:02d}"    for i in range(1, 13)]




# ── Helpers ─────────────────────────────────────────────────────────────────
def make_feed_long(mat: pd.DataFrame) -> pd.DataFrame:
    """Return FEEDER_ID|MONTH|FEEDER_LOAD_AVG (Jan..Dec complete for all feeders)."""
    need = {"SWNO", *MONTH_KEYS}
    miss = [c for c in need if c not in mat.columns]
    if miss:
        raise RuntimeError(f"monthly matrix missing columns: {miss}")

    m = mat.copy()
    m["FEEDER_ID"] = m["SWNO"].astype(str).str.strip()
    for c in MONTH_KEYS:
        m[c] = pd.to_numeric(m[c], errors="coerce").fillna(0.0)

    long = (m.melt(id_vars=["FEEDER_ID"], value_vars=MONTH_KEYS,
                   var_name="MONTH_KEY", value_name="FEEDER_LOAD_AVG")
              .assign(MONTH=lambda d: d["MONTH_KEY"].str.replace("Month_", f"{YEAR}-", regex=False))
              .drop(columns=["MONTH_KEY"]))
    complete = (pd.MultiIndex.from_product([m["FEEDER_ID"].unique(), MONTH_TAGS],
                                           names=["FEEDER_ID","MONTH"]).to_frame(index=False))
    return (complete.merge(long, on=["FEEDER_ID","MONTH"], how="left")
                    .fillna({"FEEDER_LOAD_AVG": 0.0}))

def build_graph(df_f: pd.DataFrame) -> dict[str, list[str]]:
    """Adjacency: SOURCE_LOCATION -> [DESTINATION_LOCATION...] (dedup)."""
    adj: dict[str, list[str]] = {}
    for s, d in zip(df_f["SOURCE_LOCATION"].astype(str), df_f["DESTINATION_LOCATION"].astype(str)):
        adj.setdefault(s, [])
        if d not in adj[s]:
            adj[s].append(d)
        adj.setdefault(d, [])  # ensure leaf key
    return adj

def dfs_subtree_cum(adj: dict[str, list[str]],
                    node_vec: dict[str, np.ndarray]) -> dict[str, np.ndarray]:
    """
    Post-order DFS (include-self). Each node gets a length-12 int vector:
      agg[node] = node_vec[node] + sum(agg[child]).
    """
    agg: dict[str, np.ndarray] = {}
    state: dict[str, int] = {}  # 0=unseen,1=visiting,2=done

    all_nodes = set(adj.keys())
    for kids in adj.values():
        all_nodes.update(kids)

    for root in list(all_nodes):
        if state.get(root, 0) != 0:
            continue
        stack: list[tuple[str, int]] = [(root, 0)]
        while stack:
            node, phase = stack.pop()
            if phase == 0:
                if state.get(node, 0) == 1:
                    continue  # cycle guard
                if state.get(node, 0) == 2:
                    continue
                state[node] = 1
                stack.append((node, 1))
                for ch in adj.get(node, []):
                    if state.get(ch, 0) != 2:
                        stack.append((ch, 0))
            else:
                subtotal = node_vec.get(node, np.zeros(12, dtype=np.int64)).copy()
                for ch in adj.get(node, []):
                    subtotal += agg.get(ch, np.zeros(12, dtype=np.int64))
                agg[node] = subtotal
                state[node] = 2
    return agg


# ── Main ────────────────────────────────────────────────────────────────────
def main():
    if not IN_FEDDER.exists():
        raise FileNotFoundError(f"Feeder file not found: {IN_FEDDER}")
    if not IN_MATRIX.exists():
        raise FileNotFoundError(f"Monthly matrix not found: {IN_MATRIX}")

    # 1) Feeder trace
    df = pd.read_csv(IN_FEDDER, low_memory=False)
    need = {"FEEDER_ID","SOURCE_LOCATION","DESTINATION_LOCATION"}
    miss = [c for c in need if c not in df.columns]
    if miss:
        raise RuntimeError(f"Input missing columns: {miss}")
    df["FEEDER_ID"] = df["FEEDER_ID"].astype(str).str.strip()
    if "LOCATION" not in df.columns:
        df["LOCATION"] = df["DESTINATION_LOCATION"]

    # Ensure DT_LOAD_CAL (if missing, compute from DT_LOAD with nominal denominator)
    if "DT_LOAD_CAL" not in df.columns:
        den = 11 * 0.95 * np.sqrt(3.0)
        df["DT_LOAD_CAL"] = pd.to_numeric(df.get("DT_LOAD", np.nan), errors="coerce") / den

    # 2) Fractions per feeder on UNIQUE nodes (sum to avoid double counting)
    uniq = (df[["FEEDER_ID","LOCATION","DT_LOAD_CAL"]]
            .dropna(subset=["LOCATION"])
            .groupby(["FEEDER_ID","LOCATION"], as_index=False)
            .agg(DT_LOAD_CAL=("DT_LOAD_CAL","sum")))
    sum_by_fid = uniq.groupby("FEEDER_ID")["DT_LOAD_CAL"].sum().rename("SUM_DT_LOAD_CAL")
    uniq = uniq.merge(sum_by_fid, on="FEEDER_ID", how="left")
    uniq["FRACTION"] = uniq["DT_LOAD_CAL"] / uniq["SUM_DT_LOAD_CAL"].replace(0, np.nan)

    # Bring only what's needed back to df
    loc_key = "DESTINATION_LOCATION"
    df = df.merge(
        uniq.rename(columns={"LOCATION": loc_key})[["FEEDER_ID", loc_key, "SUM_DT_LOAD_CAL", "FRACTION"]],
        on=["FEEDER_ID", loc_key],
        how="left",
        validate="many_to_one"
    )

    # 3) Feeder monthly averages (from matrix)
    matrix = pd.read_csv(IN_MATRIX, low_memory=False)
    feed_long = make_feed_long(matrix)  # FEEDER_ID|MONTH|FEEDER_LOAD_AVG
    # wide copy (optional, kept for reference/output)
    feed_wide = (feed_long.pivot_table(index="FEEDER_ID", columns="MONTH",
                                       values="FEEDER_LOAD_AVG", aggfunc="mean", fill_value=0.0)
                            .reindex(columns=MONTH_TAGS, fill_value=0.0)
                            .reset_index())
    feed_wide.columns = ["FEEDER_ID"] + FEED_COLS
    df = df.merge(feed_wide, on="FEEDER_ID", how="left")

    # 4) Per-node monthly corrected loads → integer node_load_YYYY_MM (round BEFORE cumulative)
    out_nodes = (uniq[["FEEDER_ID","LOCATION","FRACTION"]]
                 .merge(feed_long, on="FEEDER_ID", how="left"))
    out_nodes["NODE_LOAD"] = out_nodes["FRACTION"].fillna(0.0) * out_nodes["FEEDER_LOAD_AVG"].fillna(0.0)

    node_wide = (out_nodes.pivot_table(index=["FEEDER_ID","LOCATION"], columns="MONTH",
                                       values="NODE_LOAD", aggfunc="sum", fill_value=0.0)
                           .reindex(columns=MONTH_TAGS, fill_value=0.0)
                           .reset_index())


    node_wide[MONTH_TAGS] = np.ceil(node_wide[MONTH_TAGS].astype(float)).astype("Int64")

    node_wide_out = node_wide.rename(columns={"LOCATION": loc_key}).copy()
    node_wide_out.columns = ["FEEDER_ID", loc_key] + NODE_COLS
    df = df.merge(node_wide_out, on=["FEEDER_ID", loc_key], how="left")

    # 5) Bottom-up DFS per feeder → integer cumulative section loads (sec_cum_*)
    sec_parts: list[pd.DataFrame] = []
    for fid, gdf in df.groupby("FEEDER_ID", sort=False):
        # Graphfor fid, gdf in df.groupby("FEEDER_ID", sort=False):
        # Graph
        adj = build_graph(gdf[["SOURCE_LOCATION","DESTINATION_LOCATION"]])

        # Node vectors for this feeder (Int64 arrays length 12)
        sub_nodes = node_wide[node_wide["FEEDER_ID"] == fid]
        node_vec: dict[str, np.ndarray] = {str(loc): sub_nodes.loc[sub_nodes["LOCATION"] == loc, MONTH_TAGS]
                                                       .to_numpy(dtype="int64")
                                                       .reshape(-1, 12)[0]
                                           for loc in sub_nodes["LOCATION"].astype(str).unique()}
        # ensure every node in the graph has a vector
        all_nodes = set(adj.keys())
        for kids in adj.values():
            all_nodes.update(kids)
        for n in all_nodes:
            node_vec.setdefault(n, np.zeros(12, dtype=np.int64))

        # Aggregate subtree monthly totals (include self), then cumulative Jan→Dec
        agg = dfs_subtree_cum(adj, node_vec)
        rows = []
        for loc, vec in agg.items():
            cum = np.cumsum(vec, dtype=np.int64)
            rows.append(
                {"FEEDER_ID": fid, "DESTINATION_LOCATION": loc, **{
                    f"sec_cum_{m.replace('-', '_')}": int(cum[j]) for j, m in enumerate(MONTH_TAGS)
                }}
            )
        sec_parts.append(pd.DataFrame(rows))

    sec_all = pd.concat(sec_parts, ignore_index=True) if sec_parts else pd.DataFrame(
        columns=["FEEDER_ID","DESTINATION_LOCATION"] + CUM_COLS
    )
    df = df.merge(sec_all, on=["FEEDER_ID","DESTINATION_LOCATION"], how="left")

    # 6) Write
    df.to_csv(OUT_FILE, index=False)
    print(f"✓ Wrote with integer node_load_* and integer sec_cum_* → {OUT_FILE}")

    # Peek
    cols = ["FEEDER_ID","DESTINATION_LOCATION","node_load_2024_01","sec_cum_2024_03"]
    print(df[[c for c in cols if c in df.columns]].head(12).to_string(index=False))


if __name__ == "__main__":
    main()


✓ Wrote with integer node_load_* and integer sec_cum_* → /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv
FEEDER_ID   DESTINATION_LOCATION  node_load_2024_01  sec_cum_2024_03
    10205 1S-MH-MU-ZSC-CL09-2382                  8             52.0
    10205 1S-MH-MU-ZSC-CL09-2383                  8             27.0
    10205 1S-MH-MU-ZSC-CL09-2723                  6            114.0
    10205 1S-MH-MU-ZSC-CL09-2366                  2             32.0
    10205 1S-MH-MU-ZSC-CL09-2348                  5             26.0
    10205 1S-MH-MU-ZSC-CL09-2019                  3              9.0
    10205 1S-MH-MU-ZSC-CL09-2074                  3             63.0
    10205 1S-MH-MU-ZSC-CL06-3404                  0             29.0
    10205 1S-MH-MU-ZSC-CL06-3489                  3             29.0
    10205 1S-MH-MU-ZSC-CL06-2088                  6             20.0
    10205 1S-MH-MU-ZSC-CL09-3373                  1       

In [4]:
df= pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv")
col = df['FEEDER_ID'].unique()
print(len(col))

1099
